# คำเเนะนำ

*   ควรจะโหลดข้อมูลให้ครบ
*   เชื่อม google drive




**สิ่งที่จะใช้**

*    Embedding model คือ BGE - M3
*    vector database  ของ Chroma db
*    RAG ของ langchain


ทำการเชื่อม google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


โหลดสิ่งที่จำเป็น

In [ ]:
!pip install transformers torch chromadb
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q langchain transformers chromadb requests
!pip install -q langchain chromadb requests

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q langchain transformers chromadb requests
!pip install -q langchain chromadb requests


ใช้โมเดล BGE - M3

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
import numpy as np

# ✅ ใช้โมเดล BGE-M3
model_name = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# ✅ mean pooling function สำหรับ BGE
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return (token_embeddings * input_mask_expanded).sum(1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# ✅ ฟังก์ชันสร้าง embedding จากข้อความเดียว
def embed_text(text):
    # ถ้าจะใช้ตามแนวทาง BGE: prepend ด้วย "passage: " หรือ "query: " ตามบริบท
    text = "passage: " + text.strip()

    encoded_input = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)

    embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    return embedding.squeeze().cpu().numpy()

# ✅ ฟังก์ชันสร้าง embedding สำหรับชุดคำแนะนำ
def embed_recommendations(categorized_recommendations):
    for idx, rec in enumerate(categorized_recommendations, start=1):
        text = rec['text']
        embedding = embed_text(text)
        rec['embedding'] = embedding.tolist()
        print(f"✅ ข้อความที่ {idx} ได้รับการ embed แล้ว")
    return categorized_recommendations

# ✅ บันทึกไฟล์ JSON
def save_embeddings_to_file(data, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ บันทึกไฟล์ที่มีข้อมูล embedding ลงใน: {output_file_path}")

# ✅ โหลดไฟล์คำแนะนำ
def read_recommendations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# ✅ Main
file_path = '/content/drive/MyDrive/LockLearn/recommendations_with_categories.json'  # 🔄 เปลี่ยนได้ตาม path จริง
recommendations = read_recommendations(file_path)

recommendations_with_embeddings = embed_recommendations(recommendations)

embedding_output_file = '/content/drive/MyDrive/LockLearn/recommendations_with_embeddings_BGE_M3.json'
save_embeddings_to_file(recommendations_with_embeddings, embedding_output_file)

print("✅ เสร็จสิ้นการทำ embedding และบันทึกข้อมูลลงไฟล์แยก")


สร้าง vector database

In [ ]:
import chromadb
import json

# 🚀 โหลดไฟล์ embeddings ที่สร้างจาก BGE M3
embedding_file = '/content/drive/MyDrive/LockLearn/recommendations_with_embeddings_BGE_M3.json'
with open(embedding_file, 'r', encoding='utf-8') as f:
    recommendations_with_embeddings = json.load(f)

# 🚀 เชื่อมต่อกับ Chroma DB v5
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v5'
client = chromadb.PersistentClient(path=db_path)

# 🔄 ลบ Collection เก่า (ถ้ามีอยู่แล้ว)
if "recommendations" in [col.name for col in client.list_collections()]:
    client.delete_collection(name="recommendations")

# 🚀 สร้าง Collection ใหม่
collection = client.get_or_create_collection(name="recommendations")

# 🚀 เพิ่มข้อมูล embeddings ลง Chroma DB
for i, rec in enumerate(recommendations_with_embeddings):
    collection.add(
        documents=[rec['text']],
        embeddings=[rec['embedding']],
        metadatas=[{"category": rec.get('category', 'Unknown')}],
        ids=[f"rec_{i}"]
    )

print("✅ เก็บคำแนะนำทั้งหมดที่มี embedding จาก BGE M3 ลง Chroma DB v5 ใน Google Drive เรียบร้อย")
